# Práctica Clase 6

1) Cargar el dataset "Emisiones_CO2.csv" provisto en la clase 2 en un Dataframe de Pandas, quitar los registros que contengan valores faltantes e implementar una nueva columna, que contenga el resultado de una función Hash aplicada sobre el campo "Código de País" y se denomine "Clave_Hash".<br>
Consideraciones: Se puede utilizar la función provista.


In [47]:
def hash_function(key):
    return sum(index * ord(character) for index, character in enumerate(repr(key), start=1))  # Nota recordar que el star indica en que indice inicia 
                                                                                              # ord() se usa para crear un codigo para el caracter

In [48]:
print(hash_function('pato'))
print(hash_function('tapo'))

1807
1799


In [49]:
import pandas as pd

In [50]:
df = pd.read_csv('Emisiones_CO2.csv',sep='|',decimal=',', encoding='latin-1',thousands='.') #para hacer la transformacion de los millones usar thousands='.'
df 

,Código de país,Nombre del país,Región,Año,CO2 (kt),CO2 per cápita (toneladas métricas)
0,ABW,Aruba,América Latina y Caribe,1960,NaN,NaN
1,ABW,Aruba,América Latina y Caribe,1961,NaN,NaN
2,ABW,Aruba,América Latina y Caribe,1962,NaN,NaN
3,ABW,Aruba,América Latina y Caribe,1963,NaN,NaN
4,ABW,Aruba,América Latina y Caribe,1964,NaN,NaN
...,...,...,...,...,...,...
11122,ZWE,Zimbabue,África subsahariana,2007,9732.22,0.731867
11123,ZWE,Zimbabue,África subsahariana,2008,7682.37,0.569255
11124,ZWE,Zimbabue,África subsahariana,2009,8239.75,0.600521
11125,ZWE,Zimbabue,África subsahariana,2010,9028.15,0.646073


2. A partir del Dataframe creado en el punto 1, construir uno nuevo, que contenga solo los valores distintos de la tupla "Clave_Hash", "Código de País" , "Nombre de país" y "Región". Quitando luego del dataframe original los campos "Código de País" , "Nombre de país" y "Región"

In [51]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 6 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Código de país                       11127 non-null  object 
 1   Nombre del país                      11127 non-null  object 
 2   Región                               11127 non-null  object 
 3   Año                                  11127 non-null  int64  
 4   CO2 (kt)                             9032 non-null   float64
 5   CO2 per cápita (toneladas métricas)  9029 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 521.7+ KB


In [52]:
#empiezo quitando los valores NaN
df.dropna(inplace=True)

#para trabajar de mejor manera las columnas se recomienda cambiar las mayusculas las tildes y los acentos 

df.rename(columns={'Código de país':'cod_pais',
                   'Nombre del país': 'nom_pais',
                   'Región': 'region',
                   'Año': 'year',
                   'CO2 (kt)': 'co2_kt',
                   'CO2 per cápita (toneladas métricas)': 'co2_per_cap'} , inplace=True)

In [53]:
#df['clave_hash'] = df['cod_pais'].apply(hash_function)   
#como el cod_pais es muy corto habra colision en la clave hash, por eso se intenta de hacer mas lago el codigo
df['clave_hash'] = (df['cod_pais'] + df['nom_pais']).apply(hash_function)   


In [54]:
#ahora busco los codigos para cada pais unico, eliminando los duplicados

df_unicos = df[['cod_pais','nom_pais', 'region', 'clave_hash']].drop_duplicates(ignore_index=True) #de esta forma no son los indeces de la otra df, si no que se reinician
df_unicos.index += 1  #para que no empiece en 0 
df_unicos

,cod_pais,nom_pais,region,clave_hash
1,ABW,Aruba,América Latina y Caribe,4590
2,AND,Andorra,Europa y Asia Central,6949
3,AFG,Afganistán,Sur de Asia,12844
4,AGO,Angola,África subsahariana,5663
5,ALB,Albania,Europa y Asia Central,6671
...,...,...,...,...
195,YEM,Yemen,Oriente Medio y Norte de África,4734
196,ZAF,Sudáfrica,África subsahariana,10519
197,COD,Congo (República Democrática),África subsahariana,58966
198,ZMB,Zambia,África subsahariana,5637


In [55]:
#ya para terminar retorno el df que identifique los paices solo por su clave hash

df.drop(['cod_pais','nom_pais', 'region'], inplace= True , axis= 1)  
# axis es para especificar si se opera en Fila o Columnas: Fubtol Club -> F=0  C=1

df

,year,co2_kt,co2_per_cap,clave_hash
26,1986,179.683,2.868319,4590
27,1987,447.374,7.234964,4590
28,1988,612.389,10.026508,4590
29,1989,649.059,10.634733,4590
30,1990,1840.830,29.620165,4590
...,...,...,...,...
11122,2007,9732.220,0.731867,8242
11123,2008,7682.370,0.569255,8242
11124,2009,8239.750,0.600521,8242
11125,2010,9028.150,0.646073,8242


Ahora recordar que para el hash no siempre se crearan codigos unicos para cada pais se verifica con value_count

In [56]:
df_unicos['clave_hash'].value_counts()  # se aprecia que solo hay una colision 

clave_hash
11429    2
4590     1
7024     1
8407     1
5777     1
        ..
5504     1
15402    1
23706    1
5577     1
8242     1
Name: count, Length: 198, dtype: int64